In [6]:
from sedfitter.sed import SEDCube
import sedfitter
from sedfitter.extinction import Extinction
from sedfitter.source import Source
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
import astropy.units as u
import os

In [2]:
models = os.listdir('../data/galaxySEDs/')
if '.DS_Store' in models: models.remove('.DS_Store')
models = [x for x in models if 'norm' not in x]
print(len(models))

32


In [3]:
extinction = Extinction.from_file('../data/raw/whitney.r550.par')

In [4]:
apertures = [3., 3., 3., 3., 3., 3., 3., 3.] * u.arcsec

In [5]:
# Still want to replace this with Convolved filters
filters = [1.235 * u.um, 1.662 * u.um, 2.159 * u.um, 3.55 * u.um, 4.493 * u.um, 5.580 * u.um, 7.650 * u.um, 24.28 * u.um]

In [ ]:
#Testing first SESNA data point with fitting algorithm
fitter = Fitter(filters, apertures, '../data/galaxySEDs/Ly_break',
              extinction_law=extinction,
              distance_range=[0.869, 0.961] * u.kpc,
              av_range=[0., 40.], remove_resolved=True)


In [ ]:
sourcepath = '../data/raw/SESNA_sources_normalized'
sourcefile = open(sourcepath,'r')
s = Source()
s = Source.from_ascii(sourcefile.readline())

info = fitter.fit(s)








sourcefile.close()